# 2D benchmark for $\gamma=100$ beams using field-only kernel

In this notebook, 2D (i.e., in the bending plane) logitudinal and transverse CSR fields of a beam is benchmarked with those from a convolution with the beam density profile (obtained from a separate python script). The results are generated using the field-only kernel and the input deck used is input/test_beam_g100_l200.py. The fine time step used is to ensure that the transition of the trajectory between vacuum and dipole regions is resolved. It is recommended to use enough simulation particles (e.g., 1e6) to reduce the noise in the field. Note that the data loader in CosyrAnalyze automatically normalizes the data by $\gamma^4$.

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
from scipy import signal
from numpy import linalg 

import CosyrAnalyze 
import seaborn as sns

# plot setting 
mpl.rcParams['font.size'] = 16
mpl.rcParams["mathtext.default"] = "it"
mpl.rcParams['mathtext.fontset'] = 'cm'

#sns.plotting_context("poster")
sns.set_style("white")
#sns.axes_style({'font.family':'serif', 'font.serif':'Arial'})
sns.set_style({'font.family':'sans-serif', 'font.sans-serif':'Latin Modern Roman', "xtick.direction": "in","ytick.direction": "in"})
#print("Using following seaborn style:")
#print(sns.axes_style())

# scientific notation for numbers in axis labels
def sci_format(limits=(-2, 3)):
    fmt = mpl.ticker.ScalarFormatter(useMathText=True, useOffset=False)
    fmt.set_powerlimits((limits[0], limits[1]))
    return (fmt)

def ellipse(xc, yc, hv, sample=100):
    """return x, y for an ellipse with center (xc, yc) and axes given by hv;
    number of sampling points along the ellipse";
    """
    theta = np.linspace(0, 2.0*np.pi, sample)
    ellip_x = hv[0]*np.cos(theta)+xc
    ellip_y = hv[1]*np.sin(theta)+yc
    
    return (ellip_x, ellip_y)

# Cosyr result analysis

## Load wavefronts and mesh

In [2]:
gamma = 100
nalpha = 3001
nchi = 51
step = 99999
particle_id = 0
ndirs = 0
R_bend = 1.0 # meter
beam_charge = 0.01*1e-9  # Coulomb
N = beam_charge/1.6e-19  # number of real particles in the beam
sigma_s = 200.0e-6/R_bend * gamma**3.0 / 6.0
sigma_x = 200.0e-6/R_bend * gamma**2.0 / 6.0
s_axis_range = 12.0 * sigma_s
x_axis_range = 12.0 * sigma_x

beta = np.sqrt(1.0-gamma**(-2.0))
e_cgs = 4.8032e-10 # statCoul
R_cgs = R_bend*100 # cm

cosyr_result_root = "/global/homes/h/huangck/CFS/cosyr_results/test_beam_g100_l200_d200/"

In [3]:
# subcycle only result
output_path = cosyr_result_root + "test_beam_g100_l200_d200_sub_pm200_1e5steps_3000x50_10Mpart"
cosyr_2D_g100_l200_sub_only=CosyrAnalyze.CosyrAnalyze(gamma, output_path, charge=beam_charge, pid=particle_id, step = step, load_data_all=0, wf_xy_rotate=0, )
cosyr_2D_g100_l200_sub_only.load_wavefronts()
cosyr_2D_g100_l200_sub_only.load_cmesh()

setting gamma to  100
setting data_dir to  /global/homes/h/huangck/CFS/cosyr_results/test_beam_g100_l200_d200/test_beam_g100_l200_d200_sub_pm200_1e5steps_3000x50_10Mpart
setting charge to  1.0000000000000001e-11
setting R_bend to  1.0
setting pid to  0
setting step to  99999
setting dt to  0.0001
setting traj_type to  2
setting load_data_all to  0
setting wf_xy_rotate to  0
setting wf_xy2polar to  0
setting p_beam to  None
setting self to  <CosyrAnalyze.CosyrAnalyze object at 0x2aaaebcf16d0>


In [4]:
# dynamic only result
output_path = cosyr_result_root + "test_beam_g100_l200_d200_dyn_1e5steps_3000x50_10Mpart"
cosyr_2D_g100_l200_dyn_only=CosyrAnalyze.CosyrAnalyze(gamma, output_path, charge=beam_charge, pid=particle_id, step = step, load_data_all=0, wf_xy_rotate=0, )
cosyr_2D_g100_l200_dyn_only.load_wavefronts()
cosyr_2D_g100_l200_dyn_only.load_cmesh()

setting gamma to  100
setting data_dir to  /global/homes/h/huangck/CFS/cosyr_results/test_beam_g100_l200_d200/test_beam_g100_l200_d200_dyn_1e5steps_3000x50_10Mpart
setting charge to  1.0000000000000001e-11
setting R_bend to  1.0
setting pid to  0
setting step to  99999
setting dt to  0.0001
setting traj_type to  2
setting load_data_all to  0
setting wf_xy_rotate to  0
setting wf_xy2polar to  0
setting p_beam to  None
setting self to  <CosyrAnalyze.CosyrAnalyze object at 0x2aaaebcf1d00>


In [5]:
# dynamic + subcycle result
output_path = cosyr_result_root + "test_beam_g100_l200_d200_dyn_sub_psi0.01_1e5steps_3000x50_10Mpart"
cosyr_2D_g100_l200_dyn_sub=CosyrAnalyze.CosyrAnalyze(gamma, output_path, charge=beam_charge, pid=particle_id, step = step, load_data_all=0, wf_xy_rotate=0, )
cosyr_2D_g100_l200_dyn_sub.load_wavefronts()
cosyr_2D_g100_l200_dyn_sub.load_cmesh()

setting gamma to  100
setting data_dir to  /global/homes/h/huangck/CFS/cosyr_results/test_beam_g100_l200_d200/test_beam_g100_l200_d200_dyn_sub_psi0.01_1e5steps_3000x50_10Mpart
setting charge to  1.0000000000000001e-11
setting R_bend to  1.0
setting pid to  0
setting step to  99999
setting dt to  0.0001
setting traj_type to  2
setting load_data_all to  0
setting wf_xy_rotate to  0
setting wf_xy2polar to  0
setting p_beam to  None
setting self to  <CosyrAnalyze.CosyrAnalyze object at 0x2aaaebcf1820>


## Load steady-state convolution result

In [6]:
normalize_for_per_particle = True

# resolution used in the steady-state convolution calculation, in unit of sigma
depsilon = 0.06; dalpha = 0.0006
#depsilon = 0.24; dalpha = 0.004

if (normalize_for_per_particle) : 
    scaling_coeff = - depsilon*dalpha  * (1 / 2.0 / np.pi) / gamma**4.0
else: 
    scaling_coeff = - depsilon*dalpha  * (N / 2.0 / np.pi) / gamma**4.0


# lw-csr result for l200 d200 tilted beam
lwcsr_root = "/global/homes/h/huangck/Work/cosyr_benchmark/"
# 20000x200
es_lwcsr = np.load(lwcsr_root + "es_beam-csr-rad-l-200-d-200-nx-200-nalpha-20000-depsilon-0.06-dalpha-0.0006-gamma-100-theta-0.3-proj-angle-np.zeros_like(alphav).npy")
ex_lwcsr = np.load(lwcsr_root + "es_beam-csr-rad-l-200-d-200-nx-200-nalpha-20000-depsilon-0.06-dalpha-0.0006-gamma-100-theta-0.3-proj-angle--np.ones_like(alphav)*np.pi_2.0.npy")
bz_lwcsr = np.load(lwcsr_root + "es_beam-csr-rad-Bz-l-200-d-200-nx-200-nalpha-20000-depsilon-0.06-dalpha-0.0006-gamma-100-theta-0.3-proj-angle--np.ones_like(alphav)*np.pi_2.0.npy")

# 3000x50 
#es_lwcsr = np.load(lwcsr_root + "es_beam-csr-rad-l-200-d-200-nx-50-nalpha-3000-depsilon-0.24-dalpha-0.004-gamma-100-theta-0.3-proj-angle-np.zeros_like(alphav).npy")
#ex_lwcsr = np.load(lwcsr_root + "es_beam-csr-rad-l-200-d-200-nx-50-nalpha-3000-depsilon-0.24-dalpha-0.004-gamma-100-theta-0.3-proj-angle--np.ones_like(alphav)*np.pi_2.0.npy")
#bz_lwcsr = np.load(lwcsr_root + "es_beam-csr-rad-Bz-l-200-d-200-nx-50-nalpha-3000-depsilon-0.24-dalpha-0.004-gamma-100-theta-0.3-proj-angle--np.ones_like(alphav)*np.pi_2.0.npy")


es_lwcsr *= scaling_coeff
ex_lwcsr *= scaling_coeff
bz_lwcsr *= scaling_coeff
f_perp_lwcsr = ex_lwcsr - beta*bz_lwcsr

nx_lwcsr = es_lwcsr.shape[0]
nalpha_lwcsr = es_lwcsr.shape[1]
alpha_axis_lwcsr = np.linspace(-200, 200, nalpha_lwcsr)
chi_axis_lwcsr = np.linspace(-2, 2, nx_lwcsr)

In [7]:
num_subplots=3
fig_lwcsr, axes_lwcsr = plt.subplots(num_subplots,1, figsize=[5.5,4*num_subplots], dpi=120)

if (normalize_for_per_particle) :
    fld_unit = r"[eN\gamma^4/R^2]"
else:
    fld_unit = r"[e\gamma^4/R^2]"
    
fld_names = {"Bz":r"B_{z}^{rad}",
             "Ex":r"E_{x}^{rad}",
             "Fx":r"W_{\perp}^{rad}", 
             "Es":r"W_{s}^{rad}" }    

# longitudinal field
ax = axes_lwcsr[0]
p1=ax.imshow(es_lwcsr, extent=[alpha_axis_lwcsr[0], alpha_axis_lwcsr[-1],
                               chi_axis_lwcsr[0], chi_axis_lwcsr[-1]], 
             cmap="jet", aspect="auto", origin="lower")
p1.set_clim(-2e-3,6e-3)
ax.set_xlabel(r'$\tilde{\alpha} = \alpha \gamma^3$')
ax.set_ylabel(r'$\tilde{\chi} = \chi \gamma^2$')
plt.colorbar(p1, ax=ax, label = r"$" + fld_names["Es"] + fld_unit + r"$")

# transverse electric field
ax = axes_lwcsr[1]
p2=ax.imshow(ex_lwcsr, extent=[alpha_axis_lwcsr[0], alpha_axis_lwcsr[-1],
                               chi_axis_lwcsr[0], chi_axis_lwcsr[-1]], 
             cmap="jet", aspect="auto", origin="lower")
#p2.set_clim(-2e-3,6e-3)
ax.set_xlabel(r'$\tilde{\alpha} = \alpha \gamma^3$')
ax.set_ylabel(r'$\tilde{\chi} = \chi \gamma^2$')
plt.colorbar(p2, ax=ax, label = r"$" + fld_names["Ex"] + fld_unit + r"$")

# transverse wakefield
ax = axes_lwcsr[2]
p3=ax.imshow(f_perp_lwcsr, extent=[alpha_axis_lwcsr[0], alpha_axis_lwcsr[-1],
                               chi_axis_lwcsr[0], chi_axis_lwcsr[-1]], 
             cmap="jet", aspect="auto", origin="lower")
#p3.set_clim(-2e-3,6e-3)
ax.set_xlabel(r'$\tilde{\alpha} = \alpha \gamma^3$')
ax.set_ylabel(r'$\tilde{\chi} = \chi \gamma^2$')
plt.colorbar(p3, ax=ax, label = r"$" + fld_names["Fx"] + fld_unit + r"$")

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2D plot

In [8]:
#---------------------------------------------

data_to_plot = [ cosyr_2D_g100_l200_sub_only,
                 cosyr_2D_g100_l200_dyn_only,
                 cosyr_2D_g100_l200_dyn_sub
                 ]

fld_component="Es"

titles = [r'sub. only',
          r'dyn. only',
          r'dyn. + sub.',
          r'',
            ]

do_gradient = False                                                                                                                                                                  

convert_unit = False # convert to V or V/m

normalize_for_per_particle = True

data_limit = []# -2e-3, 6e-3]

#---------------------------------------------

num_subplots = len(data_to_plot)
fig, axes = plt.subplots(num_subplots,1, figsize=[5.5,4*num_subplots], dpi=120)

alpha_axis = gamma**3.0 * np.linspace(data_to_plot[0].xlim_cmesh[0], data_to_plot[0].xlim_cmesh[1], nalpha)

fld_names = {"Bz":r"B_{z}^{rad}",
             "Ex":r"E_{x}^{rad}",
             "Fx":r"W_{\perp}^{rad}", 
             "Es":r"W_{s}^{rad}" }

if (do_gradient):             
    if (convert_unit):
        fld_unit = r"[V/m]"
    else:
        if (normalize_for_per_particle) :
            fld_unit = r"[eN\gamma^4/R^2]"
        else:
            fld_unit = r"[e\gamma^4/R^2]"
else:
    if (convert_unit):
        fld_unit = r"[V]"
    else:
        if (normalize_for_per_particle) :
            fld_unit = r"[eN\gamma^4/R]"
        else:
            fld_unit = r"[e\gamma^4/R]"

i_plot = 0
for data_obj in data_to_plot:
    data_components = {"Bz":data_obj.cmesh_fld1,
                       "Ex":data_obj.cmesh_fld2,
                       "Fx":data_obj.cmesh_fld2- beta*data_obj.cmesh_fld1, 
                       "Es":data_obj.cmesh_fld3 }
    data = (data_components[fld_component]).reshape([-1,nchi]).T.copy() 
    
    if (normalize_for_per_particle) : data = data / N
    
    if (convert_unit) : data *= gamma**4.0 * e_cgs/R_cgs * 3.0e2 # -> statVolt -> Volt 
        
    if (do_gradient) : 
        data = np.gradient(data, alpha_axis/gamma**3.0, axis=1)
        
        if (convert_unit) : 
            data /= R_bend  # in unit of V/m
        
    if (num_subplots>1) : 
        ax = axes[i_plot]
    else :
        ax= axes
        
    img = ax.imshow(data,aspect="auto",cmap="jet", origin="lower", \
                    extent=[data_obj.xlim_cmesh[0]*gamma**3.0, data_obj.xlim_cmesh[1]*gamma**3.0, data_obj.ylim_cmesh[0]*gamma**2.0, data_obj.ylim_cmesh[1]*gamma**2.0] ,\
                    interpolation="bilinear")

    if (data_limit !=[]) :
        img.set_clim(data_limit)
    ax.title.set_text(titles[i_plot])
    #ax.set_xlim(-5e-6,5e-6)
    #ax.set_ylim(-2e-4,2e-4)

    cb = plt.colorbar(img, ax=ax, format=sci_format())
    fld_name = fld_names[fld_component]
    cb.set_label(r"$" + fld_name + "\ " + fld_unit + "$", usetex=False)

    ax.xaxis.set_major_formatter(sci_format())
    ax.yaxis.set_major_formatter(sci_format())

    # hide offset on the axis and move it to the label
    ax.figure.canvas.draw()
    ax.xaxis.offsetText.set_visible(False)
    ax.yaxis.offsetText.set_visible(False)

    xoffset = ax.xaxis.get_major_formatter().get_offset()
    yoffset = ax.yaxis.get_major_formatter().get_offset()
    if (not xoffset=="") : xoffset = " [" + xoffset + "]"
    if (not yoffset=="") : yoffset = " [" + yoffset + "]"
    #ax.set_xlabel(r'$x\prime$' + " [" + xoffset + "]")
    #ax.set_ylabel(r'$y\prime$' + " [" + yoffset + "]")
    ax.set_xlabel(r'$\tilde{\alpha} = \alpha \gamma^3$' + xoffset)
    ax.set_ylabel(r'$\tilde{\chi} = \chi \gamma^2$' + yoffset)

    # overlay beam envelope
    beam_env_x, beam_env_y = ellipse(0, 0, [3*sigma_s,3*sigma_x])
    ax.plot(beam_env_x, beam_env_y, 'c--') 
    
    i_plot += 1
    
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Lineout comparison

In [9]:
#---------------------------------------------

data_to_plot = [ cosyr_2D_g100_l200_sub_only,
                 cosyr_2D_g100_l200_dyn_only,
                 cosyr_2D_g100_l200_dyn_sub
                 ]

fld_component="Es"

titles = [r'sub. only',
          r'dyn. only',
          r'dyn. + sub.'
            ]

do_gradient = False                                                                                                                                                                  

convert_unit = False # convert to V or V/m

normalize_for_per_particle = True

data_enable = [True, True, True, True, True, False]
line_style = ["-", "-."] * 4

axis_shifts = [0,
               0,
               0,
               0,
              ]

do_smoothing = False

lineout_loc = -1.5 # in sigma

pycsr3d_unit = False 

save_data = False

#---------------------------------------------

fld_names = {"Bz":r"B_{z}^{rad}",
             "Ex":r"E_{x}^{rad}",
             "Fx":r"W_{\perp}^{rad}", 
             "Es":r"W_{s}^{rad}" }

if (pycsr3d_unit) : 
    # the conversion factor is Q/(4*pi*epsilon_0) = N * r_e * mc^2[eV] 
    pycsr3d_factor = beam_charge/4/np.pi/8.85e-12 
    print('pycsr3d conversion factor = ', pycsr3d_factor)

alpha_axis = gamma**3.0 * np.linspace(data_to_plot[0].xlim_cmesh[0], data_to_plot[0].xlim_cmesh[1], nalpha)

num_subplots = len(data_to_plot)
fig, axes = plt.subplots(1,1, figsize=[5.5,4], dpi=120)

ax = axes
i_plot = 0
for data_obj in data_to_plot:
    axis_shift = axis_shifts[i_plot]
    alpha_axis_sigma = (alpha_axis + axis_shift)/sigma_s
    if data_enable[i_plot] == False: 
        i_plot +=1
        continue

    data_components = {"Bz":data_obj.cmesh_fld1,
                       "Ex":data_obj.cmesh_fld2,
                       "Fx":data_obj.cmesh_fld2- beta*data_obj.cmesh_fld1, 
                       "Es":data_obj.cmesh_fld3 }   
    data = (data_components[fld_component]).reshape([-1,nchi]).T.copy() 
    
    print("i_plot, data shape = ", i_plot, data.shape)
    nx = data.shape[0]

    if (normalize_for_per_particle) : data = data / N
    
    # for potential
    if (convert_unit) : data *= gamma**4.0 * e_cgs/R_cgs * 3.0e2 # -> statVolt -> volt 
    # for field
    #if (convert_unit) : data *= gamma**4.0 * e_cgs/R_cgs/R_cgs * 3.0e4 # -> statVolt/cm -> volt/m 
        
    if (do_gradient) : 
        data = np.gradient(data, alpha_axis/gamma**3.0, axis=1)
        
        if (convert_unit) : 
            data /= R_bend # now in V/m
            if (pycsr3d_unit) : data /=pycsr3d_factor
    else :    
        if (convert_unit) : 
            if (pycsr3d_unit) : data /= - pycsr3d_factor
                
    mesh_fld_sm = data[int(nx/2+nx/12*lineout_loc),:] 
    
    sm_label = ""
    if (do_smoothing) : 
        mesh_fld_sm=signal.savgol_filter(mesh_fld_sm, 101, 3)
        sm_label = ", sm"
    
    img = ax.plot(alpha_axis, mesh_fld_sm, ls=line_style[i_plot], lw=1.5, label=r"CoSyR" + sm_label + ", " + titles[i_plot])
    if (save_data) :
        path=data_obj.data_dir
        np.savetxt(path+"/longitudinal_wake_no_smoothing_lineout_g500_10x10um.csv", mesh_fld_sm, delimiter=",")
        np.savetxt(path+"/longitudinal_wake_lineout_axis_g500_10x10um.csv", alpha_axis_sigma, delimiter=",")
        
    i_plot += 1


# plot convolution result
axis_shift = 0

lwcsr_components = {"Bz":bz_lwcsr,
                   "Ex":ex_lwcsr,
                   "Fx":f_perp_lwcsr, 
                   "Es":es_lwcsr }  
data_lwcsr = lwcsr_components[fld_component]

ax.plot(alpha_axis_lwcsr, data_lwcsr[int(nx_lwcsr/2+nx_lwcsr/12*lineout_loc),:], lw=1.5, label=r"LW-CSR")
#ax.plot(axis_lwcsr, es_lwcsr[int(25+50/12*0),:], lw=2, label=r"LW-CSR, $+3\sigma$")
#ax.plot(axis_lwcsr_3000x50 + axis_shift, es_lwcsr_3000x50_tilt[np.int(25+50/12*3),:]/N, lw=2, label=r"LW-CSR (3000x50), $-3\sigma$")
#ax.plot(axis_lwcsr_20000x200 + axis_shift, es_lwcsr_20000x200_notilt[np.int(100+200/12*3),:]/N, lw=2, label=r"LW-CSR (20000x200), $-3\sigma$")
#ax.plot(axis_lwcsr, es_lwcsr[np.int(100-200/12*3),:], lw=2, label=r"LW-CSR, $-3\sigma$")
#ax.plot(axis_1e5, beam_fld_1e5*np.max(np.abs(mesh_fld_sm)), lw=2, label=r"analytic")
#plt.plot(axis[::sample_interval],beam_shape[::sample_interval]*2e5, ls="dotted", lw=1.5, label=r"beam")

# plot labels and format
ax.xaxis.set_major_formatter(sci_format())
ax.yaxis.set_major_formatter(sci_format())

lineout_label = r"(\chi=" + str(lineout_loc)+"\sigma)"
ax.set_xlabel(r"$\tilde{\alpha} = \alpha \gamma^3$", usetex=False)
if (convert_unit) : 
    if (pycsr3d_unit) :
       ax.set_ylabel(r"$" + fld_names[fld_component] + lineout_label + r"\ [1/m^2]$", usetex=False) 
    else : # in V/m
       ax.set_ylabel(r"$" + fld_names[fld_component] + lineout_label + r"\ [V/m]$", usetex=False)
else :
    ax.set_ylabel(r"$" + fld_names[fld_component] + lineout_label + r"\ [eN\gamma^4/R^2]$", usetex=False)

ax.set_xlim(-150,150)
plt.legend(loc="best", prop={'size': 8})
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

i_plot, data shape =  0 (51, 3001)
i_plot, data shape =  1 (51, 3001)
i_plot, data shape =  2 (51, 3001)


## Plot wavefronts

In [10]:
#--------------------------------------------------
data_to_plot = [ cosyr_2D_g100_l200_sub_only,
                 cosyr_2D_g100_l200_dyn_only,
                 cosyr_2D_g100_l200_dyn_sub,
                 ]

fld_component="Es"

titles = [r'sub. only',
          r'dyn. only',
          r'dyn. + sub.',
          r'',
            ]
data_limit = [-10, 10]

#--------------------------------------------------


fld_names = {"Bz":r"B_{z}^{rad}",
             "Ex":r"E_{x}^{rad}",
             "Fx":r"W_{\perp}^{rad}",
             "Es":r"W_{s}^{rad}" }

num_subplots = len(data_to_plot)
fig, axes = plt.subplots(num_subplots,1, figsize=[5.5,4*num_subplots], dpi=120)


i_plot = 0
for data_obj in data_to_plot:
    if (num_subplots>1) : 
        ax = axes[i_plot]
    else :
        ax= axes
        
    data_components = {"Bz":data_obj.wf_fld1,
                       "Ex":data_obj.wf_fld2,
                       "Fx":data_obj.wf_fld2 - beta*data_obj.wf_fld1,
                       "Es":data_obj.wf_fld3 }   
        
    filter = np.where((np.abs(data_obj.wf_x ) < 0.5e6) & 
                      (np.abs(data_obj.wf_y) < 1e5))
    wx = data_obj.wf_x[filter]
    wy = data_obj.wf_y[filter]
    fld = (data_components[fld_component])[filter]
    
    ax.set_title(titles[i_plot])

    sc=ax.scatter(wx, wy, c=fld, marker='o', s=5, cmap="jet")
    print("data size/min/max=", fld.size, fld.min(), fld.max())
    
    #ax.set_xlim(np.array(data_obj.xlim_cmesh)*1)
    #ax.set_ylim(np.array(data_obj.ylim_cmesh)*0.25)

    cb = plt.colorbar(sc, ax=ax, format=sci_format())
    if (data_limit !=[]) :
        sc.set_clim(data_limit)    
    cb.set_label(r"$" + fld_names[fld_component] + r"\ [e\gamma^4/R^2]$", usetex=False)

    ax.xaxis.set_major_formatter(sci_format())
    ax.yaxis.set_major_formatter(sci_format())
    ax.figure.canvas.draw()
    ax.xaxis.offsetText.set_visible(False)
    ax.yaxis.offsetText.set_visible(False)

    xoffset = ax.xaxis.get_major_formatter().get_offset()
    yoffset = ax.yaxis.get_major_formatter().get_offset()
    ax.set_xlabel(r'$x\prime$' + " [" + xoffset + "]")
    ax.set_ylabel(r'$y\prime$' + " [" + yoffset + "]")
        
    i_plot +=1
    
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

data size/min/max= 303000 -2.89928235375 5.1681856475700005
data size/min/max= 95000 -8.22434168982 11.6543428293
data size/min/max= 244853 -8.22434168982 11.6543428293
